In [1]:
!pip install  gcsfs==2024.5.0 fsspec==2024.5.0 trl==0.8.6 transformers==4.44.0 datasets==2.20.0 peft==0.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/19

# Загрузка и обрабокта данных

In [2]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection
!unzip news-headlines-dataset-for-sarcasm-detection.zip

Dataset URL: https://www.kaggle.com/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection
License(s): Attribution 4.0 International (CC BY 4.0)
  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 88.6MB/s]
Archive:  news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


In [4]:
import pandas as pd
import json

data = []
with open("Sarcasm_Headlines_Dataset_v2.json", "r") as file:
    for line in file:
        data.append(json.loads(line))


df = pd.DataFrame(data)
print(df.head())
print(df.columns)

from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['headline'], df['is_sarcastic'], test_size=0.2, random_state=42
)

print(f"Train size: {len(train_texts)}, Test size: {len(test_texts)}")


   is_sarcastic                                           headline  \
0             1  thirtysomething scientists unveil doomsday clo...   
1             0  dem rep. totally nails why congress is falling...   
2             0  eat your veggies: 9 deliciously different recipes   
3             1  inclement weather prevents liar from getting t...   
4             1  mother comes pretty close to using word 'strea...   

                                        article_link  
0  https://www.theonion.com/thirtysomething-scien...  
1  https://www.huffingtonpost.com/entry/donna-edw...  
2  https://www.huffingtonpost.com/entry/eat-your-...  
3  https://local.theonion.com/inclement-weather-p...  
4  https://www.theonion.com/mother-comes-pretty-c...  
Index(['is_sarcastic', 'headline', 'article_link'], dtype='object')
Train size: 22895, Test size: 5724


In [5]:
import pandas as pd
import json

data = []
with open("Sarcasm_Headlines_Dataset_v2.json", "r") as file:
    for line in file:
        data.append(json.loads(line))

df = pd.DataFrame(data)
print(df.head())

print(df.columns)

from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['headline'], df['is_sarcastic'], test_size=0.2, random_state=42
)

print(f"Train size: {len(train_texts)}, Test size: {len(test_texts)}")


   is_sarcastic                                           headline  \
0             1  thirtysomething scientists unveil doomsday clo...   
1             0  dem rep. totally nails why congress is falling...   
2             0  eat your veggies: 9 deliciously different recipes   
3             1  inclement weather prevents liar from getting t...   
4             1  mother comes pretty close to using word 'strea...   

                                        article_link  
0  https://www.theonion.com/thirtysomething-scien...  
1  https://www.huffingtonpost.com/entry/donna-edw...  
2  https://www.huffingtonpost.com/entry/eat-your-...  
3  https://local.theonion.com/inclement-weather-p...  
4  https://www.theonion.com/mother-comes-pretty-c...  
Index(['is_sarcastic', 'headline', 'article_link'], dtype='object')
Train size: 22895, Test size: 5724


# Токенезация

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
import torch

class SarcasmDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = SarcasmDataset(train_encodings, train_labels.tolist())
test_dataset = SarcasmDataset(test_encodings, test_labels.tolist())


# Пишем самописную модель

In [8]:
import torch
import torch.nn as nn

class SarcasmDetectionModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes):
        super(SarcasmDetectionModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_ids):
        x = self.embedding(input_ids)
        _, (hidden, _) = self.lstm(x)
        output = self.fc(hidden[-1])
        return self.softmax(output)


In [31]:
# Параметры
vocab_size = len(tokenizer)
embed_dim = 128
hidden_dim = 256
num_classes = 2  # Сарказм или нет

model = SarcasmDetectionModel(vocab_size, embed_dim, hidden_dim, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


SarcasmDetectionModel(
  (embedding): Embedding(50257, 128)
  (lstm): LSTM(128, 256, batch_first=True)
  (fc): Linear(in_features=256, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)

In [32]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)


In [35]:
from torch.utils.data import DataLoader

# DataLoader для батчей
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Функция обучения
def train_model(model, train_loader, loss_fn, optimizer, device, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()

            outputs = model(input_ids)

            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")
for i in range(10):
    train_model(model, train_loader, loss_fn, optimizer, device, 1)
    evaluate_model(model, test_loader, device)


Epoch 1, Loss: 0.6920
Test Accuracy: 0.5234
Epoch 1, Loss: 0.6389
Test Accuracy: 0.7423
Epoch 1, Loss: 0.4576
Test Accuracy: 0.7966
Epoch 1, Loss: 0.3455
Test Accuracy: 0.8174
Epoch 1, Loss: 0.2596
Test Accuracy: 0.8339
Epoch 1, Loss: 0.1902
Test Accuracy: 0.8302
Epoch 1, Loss: 0.1312
Test Accuracy: 0.8344
Epoch 1, Loss: 0.0904
Test Accuracy: 0.8239


KeyboardInterrupt: 

In [36]:
def evaluate_model(model, test_loader, device):
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids)
            predictions = torch.argmax(outputs, dim=1)

            total += labels.size(0)
            correct += (predictions == labels).sum().item()

    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")

evaluate_model(model, test_loader, device)


Test Accuracy: 0.8375


# Дообученная гпт2

In [37]:
from transformers import AutoModelForSequenceClassification


model_gpt2 = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=2,
)


tokenizer.pad_token = tokenizer.eos_token
model_gpt2.resize_token_embeddings(len(tokenizer))


model_gpt2.config.pad_token_id = tokenizer.pad_token_id
model_gpt2.to(device)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

In [38]:
from torch.optim import AdamW

optimizer = AdamW(model_gpt2.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()


In [42]:
from tqdm import tqdm

def train_gpt2(model, train_loader, loss_fn, optimizer, device, epochs=3):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)

        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

            progress_bar.set_postfix({"Loss": f"{loss.item():.4f}"})

        print(f"Epoch {epoch+1} completed. Average Loss: {total_loss / len(train_loader):.4f}")
for i in range(5):
    train_gpt2(model_gpt2, train_loader, loss_fn, optimizer, device, 1)
    evaluate_gpt2(model_gpt2, test_loader, device)


Epoch 1 completed. Average Loss: 0.3820
Test Accuracy: 0.8629


Epoch 1 completed. Average Loss: 0.2454
Test Accuracy: 0.8927


Epoch 1 completed. Average Loss: 0.1896
Test Accuracy: 0.9004


Epoch 1 completed. Average Loss: 0.1471
Test Accuracy: 0.8985


KeyboardInterrupt: 

In [41]:
def evaluate_gpt2(model, test_loader, device):
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=1)

            total += labels.size(0)
            correct += (predictions == labels).sum().item()

    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")

#evaluate_gpt2(model_gpt2, test_loader, device)


In [46]:
import wandb

# Data
custom_loss = [0.6920, 0.6389, 0.4576, 0.3455, 0.2596, 0.1902, 0.1312, 0.0904]
custom_acc =  [0.5234, 0.7423, 0.7966, 0.8174, 0.8339, 0.8302, 0.8344, 0.8239]
gpt_loss = [0.3820, 0.2454, 0.1896, 0.1471]
gpt_acc = [0.8629, 0.8927, 0.9004, 0.8985]

# Initialize the W&B run
wandb.login(key="dfb4f85b3fa4bff4d1dedfdf82aab5570bde7d0b")
run = wandb.init(project='nlp_arsen')

# Log custom model metrics
for epoch in range(len(custom_loss)):
    wandb.log({
        "epoch": epoch + 1,
        "loss": custom_loss[epoch],
        "acc": custom_acc[epoch]
    })
'''

# Log GPT metrics
for epoch in range(len(gpt_loss)):
    wandb.log({
        "epoch": epoch + 1,
        "loss": gpt_loss[epoch],
        "acc": gpt_acc[epoch]
    })
'''

# Finish the run
run.finish()


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


acc,▁▆▇█████
epoch,▁▂▃▄▅▆▇█
loss,█▇▅▄▃▂▁▁
acc,0.8239
epoch,8
loss,0.0904
